In [1]:
%load_ext dotenv
%dotenv .env

In [ ]:
from redbox.app import Redbox
from redbox.models.settings import get_settings
from redbox.models.chain import RedboxQuery, RedboxState, AISettings, ChatLLMBackend
from uuid import uuid4
import langchain

INFO:root:the parsed url is ParseResult(scheme='http', netloc='admin:Opensearch2024^@localhost:9200', path='', params='', query='', fragment='')


In [3]:
def get_state(user_uuid, prompts, documents, ai_setting):
    q = RedboxQuery(
        question=f"@newroute {prompts[-1]}",
        s3_keys=documents,
        user_uuid=user_uuid,
        chat_history=prompts[:-1],
        ai_settings=ai_setting,
        permitted_s3_keys=documents,
    )

    return RedboxState(
        request=q,
    )

In [4]:
env = get_settings()
env = env.model_copy(update={"elastic_root_index": "redbox-data-integration"})
env = env.model_copy(update={"elastic_chunk_alias": "redbox-data-integration-chunk-current"})

ai_setting = AISettings(chat_backend=ChatLLMBackend(name="anthropic.claude-3-sonnet-20240229-v1:0", provider="bedrock"))
app = Redbox(debug=False, env=env)
documents = ["natasha.boyse@digital.trade.gov.uk/ai_in_analysis.docx"]

INFO:root:Testing OpenSearch is definitely being used
INFO:opensearch:HEAD http://localhost:9200/_alias/redbox-data-integration-chunk-current [status:200 request:0.025s]
INFO:opensearch:HEAD http://localhost:9200/redbox-data-integration-chat-mesage-log [status:200 request:0.010s]
INFO:root:Testing OpenSearch is definitely being used
INFO:opensearch:HEAD http://localhost:9200/_alias/redbox-data-integration-chunk-current [status:200 request:0.004s]
INFO:opensearch:HEAD http://localhost:9200/redbox-data-integration-chat-mesage-log [status:200 request:0.003s]
/Users/noraerrouhly/Documents/redbox/redbox-core/redbox/chains/components.py:82: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import BedrockEmbeddings``.
  ret

In [5]:
questions = [
    "Summarise this document with a focus on use of Redbox by analysts. Also find more information on AI and Large language model policy on gov.uk",
    "summarise this document",
    "summarise this document. also, find the definition of large language models based on wikipedia",
    "summarise this document. also, based on the document tell me what other AI assistant tool are used by DBT apart from Redbox",
]
states = []
for question in questions:
    state = get_state(uuid4(), prompts=[question], documents=documents, ai_setting=ai_setting)
    states.append(state)

In [ ]:
langchain.debug = False
planner_outputs = []
repetition = 10
with open("Planner_results.txt", "w") as text_file:
    for state in states:
        text_file.write(
            "##########################################################################################################"
        )
        text_file.write("Question")
        text_file.write("\n\n")
        text_file.write(state.request.question)
        text_file.write("\n\n")
        for rep in range(repetition):
            for chunk in app.graph.stream(state, stream_mode="updates", subgraphs=True):
                if chunk[1].get("planner"):
                    # print(chunk[1]["planner"])
                    planner_output = chunk[1]["planner"]["messages"][-1].content
                    text_file.write("Planner Output " + str(rep))
                    text_file.write("\n\n")
                    text_file.write(planner_output)
                    text_file.write("\n\n")
                    planner_outputs.append(planner_output)

INFO:opensearch:POST http://localhost:9200/redbox-data-integration-chunk-current/_search?_source=true&scroll=5m&size=1000 [status:200 request:0.030s]
INFO:opensearch:POST http://localhost:9200/_search/scroll [status:200 request:0.006s]
INFO:opensearch:DELETE http://localhost:9200/_search/scroll [status:200 request:0.003s]
INFO:root:Testing OpenSearch is definitely being used
INFO:opensearch:HEAD http://localhost:9200/_alias/redbox-data-chunk-current [status:200 request:0.005s]
INFO:opensearch:HEAD http://localhost:9200/redbox-data-chat-mesage-log [status:200 request:0.006s]
INFO:opensearch:POST http://localhost:9200/redbox-data-chunk-current/_search [status:200 request:0.010s]
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:langchain_aws.llms.bedrock:Using Bedrock Invoke API to generate response
INFO:opensearch:POST http://localhost:9200/redbox-data-integration-chunk-current/_search?_source=true&scroll=5m&size=1000 [status:200 request:0.0